In [13]:
import requests
import time
from anthropic import Anthropic
import pandas as pd
from bs4 import BeautifulSoup
import os
from datetime import datetime
from collections import defaultdict
import logging

from selenium import webdriver
from selenium.webdriver.chrome.options import Options

chrome_options = Options()
chrome_options.add_argument('--headless')
driver = webdriver.Chrome(options=chrome_options)

In [15]:
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(f'processing_log_{timestamp}.log'),
        logging.StreamHandler()
    ]
)

error_tracker = {
    'page_fetch_errors': [],
    'translation_extraction_errors': [],
    'authority_extraction_errors': [],
    'attestation_extraction_errors': [],
    'llm_api_errors': []
}

In [16]:
def fetch_page_source(node_id):
    url = f"https://nahuatl.wired-humanities.org/node/{node_id}"
    try:
        driver.get(url)
        time.sleep(0.5)
        return driver.page_source
    except Exception as e:
        error_tracker['page_fetch_errors'].append({
            'node_id': node_id,
            'error': str(e),
            'timestamp': datetime.now().isoformat()
        })
        logging.error(f"Failed to fetch page for node {node_id}: {e}")
        return None

In [17]:
def extract_attestation_field(html, language):
    soup = BeautifulSoup(html, 'html.parser')
    language_field_map = {
        'English': 'field-name-field-additionalnotes-lang1',
        'Spanish': 'field-name-field-additionalnotes-lang2'
    }
    field_class = language_field_map.get(language)
    if not field_class:
        return None
    field_div = soup.find('div', class_=field_class)
    if field_div:
        items_div = field_div.find('div', class_='field-items') # type: ignore
        if items_div:
            return str(items_div)
    return None

def extract_authority_field(html, authority):
    soup = BeautifulSoup(html, 'html.parser')
    authority_field_map = {
        'Molina': 'field-name-field-authority1',
        'Karttunen': 'field-name-field-authority2',
        'Carochi': 'field-name-field-authority3',
        'Olmos': 'field-name-field-authority4',
        'Lockhart': 'field-name-field-authority6'
    }
    field_class = authority_field_map.get(authority)
    if not field_class:
        return None
    field_div = soup.find('div', class_=field_class)
    if field_div:
        items_div = field_div.find('div', class_='field-items') # type: ignore
        if items_div:
            return str(items_div)
    return None

def extract_translation_field(html):
    soup = BeautifulSoup(html, 'html.parser')
    field_div = soup.find('div', class_='field-name-field-translation1')
    if field_div:
        items_div = field_div.find('div', class_='field-items') # type: ignore
        if items_div:
            return str(items_div)
    return None

In [18]:
def choose_best_version(local_val, scraped_val, manual_val, source_val, field_name, node_id):
    prompt = f"""Given these four versions of a {field_name}:

LOCAL DATABASE: {local_val}

SCRAPED VERSION: {scraped_val}

MANUAL CORRECTION: {manual_val}

CURRENT PAGE SOURCE: {source_val}

Which version (local, scraped, or manual) best represents what's currently on the page source?
Respond with ONLY one word: local, scraped, or manual."""

    try:
        message = client.messages.create(
            model="claude-sonnet-4-5-20250929",
            max_tokens=10,
            messages=[{"role": "user", "content": prompt}]
        )
        return message.content[0].text.strip().lower() #type: ignore
    except Exception as e:
        error_tracker['llm_api_errors'].append({
            'node_id': node_id,
            'field_name': field_name,
            'error': str(e),
            'timestamp': datetime.now().isoformat()
        })
        logging.error(f"LLM API error for node {node_id}, field {field_name}: {e}")
        return 'error'

In [19]:
def process_all_differences():
    translation_df = pd.read_csv("./review_translation_3way_final.csv")
    authority_df = pd.read_csv("./review_authority_3way_final.csv")
    attestation_df = pd.read_csv("./review_attestation_3way_final.csv")
    
    logging.info(f"Starting processing:")
    logging.info(f"  - Translations: {len(translation_df)}")
    logging.info(f"  - Authorities: {len(authority_df)}")
    logging.info(f"  - Attestations: {len(attestation_df)}")
    
    node_tasks = defaultdict(lambda: {'translations': [], 'authorities': [], 'attestations': []})
    
    for idx, row in translation_df.iterrows():
        node_tasks[row['node_id']]['translations'].append((idx, row.to_dict()))
    
    for idx, row in authority_df.iterrows():
        node_tasks[row['node_id']]['authorities'].append((idx, row.to_dict()))
    
    for idx, row in attestation_df.iterrows():
        node_tasks[row['node_id']]['attestations'].append((idx, row.to_dict()))
    
    all_results = {
        'translations': [],
        'authorities': [],
        'attestations': []
    }
    
    total_nodes = len(node_tasks)
    processed = 0
    
    for node_id, tasks in node_tasks.items():
        try:
            html = fetch_page_source(node_id)
            
            if not html:
                for idx, row in tasks['translations']:
                    all_results['translations'].append({**row, 'llm_choice': 'error', 'source_value': None, 'error_type': 'page_fetch_failed'})
                for idx, row in tasks['authorities']:
                    all_results['authorities'].append({**row, 'llm_choice': 'error', 'source_value': None, 'error_type': 'page_fetch_failed'})
                for idx, row in tasks['attestations']:
                    all_results['attestations'].append({**row, 'llm_choice': 'error', 'source_value': None, 'error_type': 'page_fetch_failed'})
                continue
            
            for idx, row in tasks['translations']:
                source_val = extract_translation_field(html)
                if source_val:
                    choice = choose_best_version(
                        row['local_punct_fixed_value'],
                        row['scraped_value'],
                        row['manual_corrections_value'],
                        source_val,
                        "Principal English Translation",
                        node_id
                    )
                    all_results['translations'].append({**row, 'llm_choice': choice, 'source_value': source_val, 'error_type': None})
                else:
                    error_tracker['translation_extraction_errors'].append({
                        'node_id': node_id,
                        'headword': row.get('headword'),
                        'timestamp': datetime.now().isoformat()
                    })
                    logging.warning(f"Could not extract translation field for node {node_id}")
                    all_results['translations'].append({**row, 'llm_choice': 'error', 'source_value': None, 'error_type': 'extraction_failed'})
                time.sleep(0.1)
            
            for idx, row in tasks['authorities']:
                source_val = extract_authority_field(html, row['authority'])
                if source_val:
                    choice = choose_best_version(
                        row['local_punct_fixed_value'],
                        row['scraped_value'],
                        row['manual_corrections_value'],
                        source_val,
                        f"{row['authority']} citation",
                        node_id
                    )
                    all_results['authorities'].append({**row, 'llm_choice': choice, 'source_value': source_val, 'error_type': None})
                else:
                    error_tracker['authority_extraction_errors'].append({
                        'node_id': node_id,
                        'authority': row['authority'],
                        'headword': row.get('headword'),
                        'timestamp': datetime.now().isoformat()
                    })
                    logging.warning(f"Could not extract {row['authority']} field for node {node_id}")
                    all_results['authorities'].append({**row, 'llm_choice': 'error', 'source_value': None, 'error_type': 'extraction_failed'})
                time.sleep(0.1)
            
            for idx, row in tasks['attestations']:
                source_val = extract_attestation_field(html, row['language'])
                if source_val:
                    choice = choose_best_version(
                        row['local_punct_fixed_value'],
                        row['scraped_value'],
                        row['manual_corrections_value'],
                        source_val,
                        f"Attestations from sources in {row['language']}",
                        node_id
                    )
                    all_results['attestations'].append({**row, 'llm_choice': choice, 'source_value': source_val, 'error_type': None})
                else:
                    error_tracker['attestation_extraction_errors'].append({
                        'node_id': node_id,
                        'language': row['language'],
                        'headword': row.get('headword'),
                        'timestamp': datetime.now().isoformat()
                    })
                    logging.warning(f"Could not extract {row['language']} attestation field for node {node_id}")
                    all_results['attestations'].append({**row, 'llm_choice': 'error', 'source_value': None, 'error_type': 'extraction_failed'})
                time.sleep(0.1)
            
            processed += 1
            if processed % 10 == 0:
                logging.info(f"Processed {processed}/{total_nodes} nodes")
                
                pd.DataFrame(all_results['translations']).to_csv('./results_translations_interim.csv', index=False)
                pd.DataFrame(all_results['authorities']).to_csv('./results_authorities_interim.csv', index=False)
                pd.DataFrame(all_results['attestations']).to_csv('./results_attestations_interim.csv', index=False)
        
        except Exception as e:
            logging.error(f"Unexpected error processing node {node_id}: {e}")
            continue
    
    pd.DataFrame(all_results['translations']).to_csv(f'./results_translations_{timestamp}.csv', index=False)
    pd.DataFrame(all_results['authorities']).to_csv(f'./results_authorities_{timestamp}.csv', index=False)
    pd.DataFrame(all_results['attestations']).to_csv(f'./results_attestations_{timestamp}.csv', index=False)
    
    pd.DataFrame(error_tracker['page_fetch_errors']).to_csv(f'./errors_page_fetch_{timestamp}.csv', index=False)
    pd.DataFrame(error_tracker['translation_extraction_errors']).to_csv(f'./errors_translation_extraction_{timestamp}.csv', index=False)
    pd.DataFrame(error_tracker['authority_extraction_errors']).to_csv(f'./errors_authority_extraction_{timestamp}.csv', index=False)
    pd.DataFrame(error_tracker['attestation_extraction_errors']).to_csv(f'./errors_attestation_extraction_{timestamp}.csv', index=False)
    pd.DataFrame(error_tracker['llm_api_errors']).to_csv(f'./errors_llm_api_{timestamp}.csv', index=False)
    
    driver.quit()
    
    logging.info(f"\nCOMPLETE")
    logging.info(f"Translations processed: {len(all_results['translations'])}")
    logging.info(f"Authorities processed: {len(all_results['authorities'])}")
    logging.info(f"Attestations processed: {len(all_results['attestations'])}")
    logging.info(f"\nERROR SUMMARY:")
    logging.info(f"  - Page fetch errors: {len(error_tracker['page_fetch_errors'])}")
    logging.info(f"  - Translation extraction errors: {len(error_tracker['translation_extraction_errors'])}")
    logging.info(f"  - Authority extraction errors: {len(error_tracker['authority_extraction_errors'])}")
    logging.info(f"  - Attestation extraction errors: {len(error_tracker['attestation_extraction_errors'])}")
    logging.info(f"  - LLM API errors: {len(error_tracker['llm_api_errors'])}")
    
    return all_results

results = process_all_differences()

2025-10-30 17:49:32,277 - INFO - Starting processing:
2025-10-30 17:49:32,277 - INFO -   - Translations: 683
2025-10-30 17:49:32,277 - INFO -   - Authorities: 1000
2025-10-30 17:49:32,277 - INFO -   - Attestations: 744
2025-10-30 17:49:38,472 - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
2025-10-30 17:49:41,507 - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
2025-10-30 17:49:44,519 - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
2025-10-30 17:49:47,565 - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
2025-10-30 17:49:52,199 - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
2025-10-30 17:49:55,113 - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
2025-10-30 17:49:57,846 - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
2025-10-30 17:49:59,914 -

In [20]:
import sqlite3

In [34]:
local_db = sqlite3.connect('../../../data/sqLiteDb/nahuatl_processing.db')
scraped_db = sqlite3.connect('../../../data/scrapedDataDb/nahuatl.db')
manual_db = sqlite3.connect('../../../data/sqLiteDb/db_that_has_tbl_b4_citation_and_cf_extraction.db')
results_translations = pd.read_csv(f'./results_translations_20251030_174931.csv')
eng_translations_nodeid_to_llmchoice = {
    row['node_id']: row['llm_choice']
    for _, row in results_translations.iterrows()
}
results_authorities = pd.read_csv(f'./results_authorities_20251030_174931.csv')
authorities_nodeid_to_llmchoice = {
    row['node_id']: str(row['llm_choice']) + "|" + str(row['authority'])
    for _, row in results_authorities.iterrows()
}
results_attestations = pd.read_csv(f'./results_attestations_20251030_174931.csv')
attestations_nodeid_to_llmchoice = {
    row['node_id']: str(row['llm_choice']) + "|" + str(row['language'])
    for _, row in results_attestations.iterrows()
}

In [35]:
checkpoint_table = "checkpoint_scraped_manual_match_fixed_20251030"
full_checkpoint = pd.read_sql(f"SELECT * FROM [{checkpoint_table}]", local_db)
full_checkpoint['Ref_clean'] = full_checkpoint['Ref'].astype(str).str.replace(r'^(WHP-|IDIEZ-)', '', regex=True)

scraped_entries = pd.read_sql("SELECT node_id, translation_english FROM dictionary_entries", scraped_db)
scraped_entries['node_id'] = scraped_entries['node_id'].astype(str)

scraped_citations = pd.read_sql("SELECT node_id, authority_name, citation_text FROM authority_citations", scraped_db)
scraped_citations['node_id'] = scraped_citations['node_id'].astype(str)

scraped_attestations = pd.read_sql("SELECT node_id, language, attestation_text FROM attestations", scraped_db)
scraped_attestations['node_id'] = scraped_attestations['node_id'].astype(str)

manual_source = pd.read_sql("SELECT * FROM checkpoint_final_20250914", manual_db)
manual_source['Ref_clean'] = manual_source['Ref'].astype(str).str.replace(r'^(WHP-|IDIEZ-)', '', regex=True)

authority_column_map = {
    'Molina': 'Alonso de Molina',
    'Karttunen': 'Frances Karttunen',
    'Carochi': 'Horacio Carochi / English',
    'Olmos': 'Andrés de Olmos',
    'Lockhart': 'Lockhart’s Nahuatl as Written'
}

update_stats = {
    'translations': {'local': 0, 'scraped': 0, 'manual': 0, 'skipped': 0},
    'authorities': {'local': 0, 'scraped': 0, 'manual': 0, 'skipped': 0},
    'attestations': {'local': 0, 'scraped': 0, 'manual': 0, 'skipped': 0}
}

for node_id, choice in eng_translations_nodeid_to_llmchoice.items():
    if choice in ['error', 'none_match']:
        update_stats['translations']['skipped'] += 1
        continue
    
    node_id_str = str(node_id)
    mask = full_checkpoint['Ref_clean'] == node_id_str
    
    if not mask.any():
        update_stats['translations']['skipped'] += 1
        continue
    
    if choice == 'local':
        update_stats['translations']['local'] += 1
    elif choice == 'scraped':
        scraped_mask = scraped_entries['node_id'] == node_id_str
        if scraped_mask.any():
            value = scraped_entries.loc[scraped_mask, 'translation_english'].iloc[0]
            full_checkpoint.loc[mask, 'Principal English Translation'] = value
            update_stats['translations']['scraped'] += 1
        else:
            update_stats['translations']['skipped'] += 1
    elif choice == 'manual':
        manual_mask = manual_source['Ref_clean'] == node_id_str
        if manual_mask.any():
            value = manual_source.loc[manual_mask, 'Principal English Translation'].iloc[0]
            full_checkpoint.loc[mask, 'Principal English Translation'] = value
            update_stats['translations']['manual'] += 1
        else:
            update_stats['translations']['skipped'] += 1
    else:
        update_stats['translations']['skipped'] += 1

print(f"Translations updated: {update_stats['translations']}")

for node_id, choice_authority in authorities_nodeid_to_llmchoice.items():
    parts = choice_authority.split('|')
    if len(parts) != 2:
        update_stats['authorities']['skipped'] += 1
        continue
    
    choice, authority = parts
    
    if choice in ['error', 'none_match']:
        update_stats['authorities']['skipped'] += 1
        continue
    
    local_column = authority_column_map.get(authority)
    if not local_column:
        update_stats['authorities']['skipped'] += 1
        continue
    
    node_id_str = str(node_id)
    mask = full_checkpoint['Ref_clean'] == node_id_str
    
    if not mask.any():
        update_stats['authorities']['skipped'] += 1
        continue
    
    if choice == 'local':
        update_stats['authorities']['local'] += 1
    elif choice == 'scraped':
        scraped_mask = (scraped_citations['node_id'] == node_id_str) & (scraped_citations['authority_name'] == authority)
        if scraped_mask.any():
            value = scraped_citations.loc[scraped_mask, 'citation_text'].iloc[0]
            full_checkpoint.loc[mask, local_column] = value
            update_stats['authorities']['scraped'] += 1
        else:
            update_stats['authorities']['skipped'] += 1
    elif choice == 'manual':
        manual_mask = manual_source['Ref_clean'] == node_id_str
        if manual_mask.any():
            value = manual_source.loc[manual_mask, local_column].iloc[0]
            full_checkpoint.loc[mask, local_column] = value
            update_stats['authorities']['manual'] += 1
        else:
            update_stats['authorities']['skipped'] += 1
    else:
        update_stats['authorities']['skipped'] += 1

print(f"Authorities updated: {update_stats['authorities']}")

for node_id, choice_language in attestations_nodeid_to_llmchoice.items():
    parts = choice_language.split('|')
    if len(parts) != 2:
        update_stats['attestations']['skipped'] += 1
        continue
    
    choice, language = parts
    
    if choice in ['error', 'none_match']:
        update_stats['attestations']['skipped'] += 1
        continue
    
    local_column = f'Attestations from sources in {language}'
    
    node_id_str = str(node_id)
    mask = full_checkpoint['Ref_clean'] == node_id_str
    
    if not mask.any():
        update_stats['attestations']['skipped'] += 1
        continue
    
    if choice == 'local':
        update_stats['attestations']['local'] += 1
    elif choice == 'scraped':
        scraped_mask = (scraped_attestations['node_id'] == node_id_str) & (scraped_attestations['language'] == language)
        if scraped_mask.any():
            value = scraped_attestations.loc[scraped_mask, 'attestation_text'].iloc[0]
            full_checkpoint.loc[mask, local_column] = value
            update_stats['attestations']['scraped'] += 1
        else:
            update_stats['attestations']['skipped'] += 1
    elif choice == 'manual':
        manual_mask = manual_source['Ref_clean'] == node_id_str
        if manual_mask.any():
            value = manual_source.loc[manual_mask, local_column].iloc[0]
            full_checkpoint.loc[mask, local_column] = value
            update_stats['attestations']['manual'] += 1
        else:
            update_stats['attestations']['skipped'] += 1
    else:
        update_stats['attestations']['skipped'] += 1

print(f"Attestations updated: {update_stats['attestations']}")

full_checkpoint = full_checkpoint.drop(columns=['Ref_clean'])

timestamp = datetime.now().strftime("%Y%m%d")
new_table_name = f"checkpoint_llm_validated_{timestamp}"

full_checkpoint.to_sql(new_table_name, local_db, if_exists='replace', index=False)

print("\nLLM VALIDATION UPDATES APPLIED")
print(f"Total translations: {sum(update_stats['translations'].values())}")
print(f"Total authorities: {sum(update_stats['authorities'].values())}")
print(f"Total attestations: {sum(update_stats['attestations'].values())}")
print(f"\nNew checkpoint created: {new_table_name}")

Translations updated: {'local': 11, 'scraped': 651, 'manual': 21, 'skipped': 0}
Authorities updated: {'local': 22, 'scraped': 676, 'manual': 277, 'skipped': 1}
Attestations updated: {'local': 10, 'scraped': 358, 'manual': 259, 'skipped': 2}

LLM VALIDATION UPDATES APPLIED
Total translations: 683
Total authorities: 976
Total attestations: 629

New checkpoint created: checkpoint_llm_validated_20251030
